In [2]:
!pip install torch torchvision numpy matplotlib pillow pymoo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.7/72.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.8/303.8 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 7.5 MB/s eta 0:00:00


In [ ]:
import math
import os
import copy
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# pymoo (NSGA-II)
from pymoo.core.problem import Problem
from pymoo.optimize import minimize
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.termination import get_termination

from datasets import load_dataset
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.models import inception_v3, Inception_V3_Weights # Para métrica de diversidade mais avançada

# --- Adicionado: Para usar um discriminador mais robusto ---
from torchvision.models import vgg11, VGG11_Weights
# -----------------------------------------------------------

class WikiArtDataset(Dataset):
    """Wrapper para o dataset do Hugging Face para aplicar transformações."""
    def __init__(self, hf_dataset, transform):
        self.hf_dataset = hf_dataset
        self.transform = transform

    def __len__(self):
        return len(self.hf_dataset)

    def __getitem__(self, idx):
        # Garante que a imagem é RGB
        img = self.hf_dataset[idx]['image'].convert('RGB')
        return self.transform(img)

def train_discriminator(discriminator, cppn_template, device, img_size=64, out_channels=3, batch_size=32, epochs=5, drive_base_path=""):
    """
    Função para pré-treinar o discriminador com o WikiArt (reais)
    e CPPNs aleatórias (falsas).
    """
    print("Iniciando pré-treinamento do Discriminador...")

    # 1. Configurar transformações
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalização padrão de GAN
    ])

    # 2. Carregar dataset
    print("Carregando WikiArt (pode demorar na primeira vez)...")
    try:
        raw_dataset = load_dataset("huggan/wikiart", split='train')
    except Exception as e:
        print(f"Falha ao carregar dataset: {e}. Tente verificar sua conexão.")
        return discriminator # Retorna não treinado

    train_dataset = WikiArtDataset(raw_dataset, transform)
    dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

    # 3. Configurar otimizador e perda
    optimizer = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    criterion = nn.BCELoss() # Perda de Entropia Cruzada Binária

    genome_size = count_params(cppn_template)

    for epoch in range(epochs):
        for i, real_images in enumerate(dataloader):

            current_batch_size = real_images.size(0)
            real_images = real_images.to(device)

            # --- 1. Treinar com Imagens REAIS ---
            discriminator.zero_grad()
            labels = torch.full((current_batch_size, 1), 1.0, device=device) # Labels = 1

            output = discriminator(real_images)
            loss_real = criterion(output, labels)
            loss_real.backward()

            # --- 2. Treinar com Imagens FALSAS ---
            fake_images_list = []
            for _ in range(current_batch_size):
                random_genome = np.random.uniform(-5.0, 5.0, size=genome_size).astype(np.float32)
                rand_cppn = copy.deepcopy(cppn_template)
                unpack_vector_to_model(rand_cppn, random_genome)
                rand_cppn.to(device)

                img_np = cppn_generate_image(rand_cppn, size=img_size, out_channels=out_channels, device=device)

                arr = np.asarray(img_np).astype(np.float32) / 255.0
                arr = arr.transpose(2,0,1)[None, ...] # (1, C, H, W)
                fake_images_list.append(torch.from_numpy(arr))

            fake_images = torch.cat(fake_images_list, dim=0).to(device).float()

            fake_images = (fake_images - 0.5) / 0.5

            labels.fill_(0.0) # Labels = 0
            output = discriminator(fake_images.detach())
            loss_fake = criterion(output, labels)
            loss_fake.backward()

            # --- 3. Atualizar Pesos ---
            loss_d = loss_real + loss_fake
            optimizer.step()

            if i % 100 == 0:
                print(f"Epoch [{epoch+1}/{epochs}], Batch [{i+1}/{len(dataloader)}], "
                      f"Loss D: {loss_d.item():.4f} (Real: {loss_real.item():.4f}, Fake: {loss_fake.item():.4f})")

    print("Treinamento do Discriminador concluído.")
    return discriminator

# -----------------------------
# 1) CPPN (PyTorch) - ALTERADA PARA MAIOR COMPLEXIDADE
# -----------------------------
class Sin(nn.Module):
    def forward(self, x):
        return torch.sin(x)

class Gaussian(nn.Module):
    def forward(self, x):
        return torch.exp(-x**2)

class CustomAct(nn.Module): # Ativação customizada extra
    def forward(self, x):
        return torch.tanh(x) * torch.sin(x * 2) # Combinação

class CPPN(nn.Module):
    """CPPN mais complexa com mais camadas e neurônios."""
    def __init__(self, out_channels=3, hidden_size=16): # Aumentar hidden_size
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, hidden_size), # Entrada para x, y
            Sin(),                     # Ativação Sin
            nn.Linear(hidden_size, hidden_size),
            Gaussian(),                # Ativação Gaussian
            nn.Linear(hidden_size, hidden_size),
            CustomAct(),               # Nova ativação
            nn.Linear(hidden_size, out_channels),
            nn.Sigmoid()
        )

    def forward(self, coords):
        return self.net(coords)

# -----------------------------
# 2) Aux: utilitários para conversão genoma <-> parâmetros
# -----------------------------
def count_params(model):
    return sum(p.numel() for p in model.parameters())

def pack_params_to_vector(model):
    with torch.no_grad():
        params = [p.view(-1).cpu().numpy() for p in model.parameters()]
    return np.concatenate(params).astype(np.float32)

def unpack_vector_to_model(model, vector):
    pointer = 0
    with torch.no_grad():
        for p in model.parameters():
            num = p.numel()
            slice_ = vector[pointer: pointer + num]
            p.copy_(torch.from_numpy(slice_.reshape(p.shape)).to(p.device))
            pointer += num
    assert pointer == len(vector)

# -----------------------------
# 3) Discriminador mais robusto (usando VGG11 pré-treinado)
# -----------------------------
class VGGDiscriminator(nn.Module):
    def __init__(self, in_channels=3):
        super().__init__()
        # Carrega um VGG11 pré-treinado e usa suas features
        vgg_features = vgg11(weights=VGG11_Weights.IMAGENET1K_V1).features

        # O VGG espera 3 canais de entrada, então não precisamos nos preocupar com in_channels aqui,
        # mas se o input fosse diferente, teríamos que adaptar a primeira camada.
        # Para um discriminador, podemos querer que ele seja mais "raspado" para nosso domínio.
        # Vamos manter o in_channels para consistência, mas VGG está hardcoded para 3.

        # Adaptar a primeira camada se in_channels não for 3 (o que não é o caso aqui)
        # self.features = vgg_features
        # if in_channels != 3:
        #     first_conv = vgg_features[0]
        #     new_first_conv = nn.Conv2d(in_channels, first_conv.out_channels,
        #                                first_conv.kernel_size, first_conv.stride,
        #                                first_conv.padding, bias=first_conv.bias is not None)
        #     self.features[0] = new_first_conv

        # Usamos as camadas de features do VGG até um certo ponto e adicionamos nossa cabeça
        # Vamos pegar até a última camada de pooling (antes do último bloco conv)
        # Ou seja, vamos usar as features profundas do VGG
        self.features = nn.Sequential(*list(vgg_features)[:18]) # Onde 18 é geralmente até o penúltimo bloco de conv

        # A saída de self.features será algo como (B, 512, H/32, W/32).
        # Para img_size=64, H/32 = 2, W/32 = 2. Então (B, 512, 2, 2)
        # Precisamos de um Average Pool para reduzir para (B, 512, 1, 1)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.head = nn.Linear(512, 1) # VGG11 termina com 512 canais nesse ponto

    def forward(self, x):
        # Normalização para VGG (espera valores entre 0-1, normalizado com ImageNet stats)
        # No nosso caso, o input já está normalizado para [-1, 1] e é re-normalizado na função de score.
        # Não precisamos re-normalizar aqui, mas é bom ter em mente para modelos pré-treinados.

        f = self.features(x)
        f = self.avgpool(f)
        f = f.view(x.size(0), -1)
        return torch.sigmoid(self.head(f))

# -----------------------------
# 4) Geração de imagens a partir de uma CPPN
# -----------------------------
def make_coordinate_grid(size, device='cpu'):
    xs = np.linspace(-1, 1, size)
    ys = np.linspace(-1, 1, size)
    X, Y = np.meshgrid(xs, ys)
    coords = np.stack([X.ravel(), Y.ravel()], axis=-1).astype(np.float32)
    return torch.from_numpy(coords).to(device)

def cppn_generate_image(cppn_model, size=64, out_channels=3, device='cpu'): # out_channels=3
    coords = make_coordinate_grid(size, device=device)
    with torch.no_grad():
        out = cppn_model(coords)
    img = out.cpu().numpy().reshape(size, size, out_channels)
    img = (img * 255).clip(0, 255).astype(np.uint8)
    if out_channels == 1:
        img = img.squeeze(-1)
    return img

# -----------------------------
# 5) Métricas / objetivos - ALTERADA PARA MAIS SOFISTICADA
# -----------------------------

# --- Adicionado: Para usar métricas de features pré-treinadas ---
# Carrega InceptionV3 pré-treinado e remove a camada de classificação
class FeatureExtractor(nn.Module):
    def __init__(self, model_name='inception_v3', device='cpu'):
        super().__init__()
        if model_name == 'inception_v3':
            weights = Inception_V3_Weights.IMAGENET1K_V1
            model = inception_v3(weights=weights, transform_input=True)
            self.transform_input = weights.transforms() # Inclui normalização
            self.features = nn.Sequential(*list(model.children())[:-1]) # Remove a camada linear final
            self.features.eval() # Modo de avaliação para não treinar
            for param in self.features.parameters():
                param.requires_grad = False
        self.to(device)

    def forward(self, x):
        # x deve ser um tensor [0,1] aqui
        # Inception_V3_Weights.IMAGENET1K_V1.transforms espera [0,1] e faz a normalização interna
        # do nosso [-1,1] para [0,1] e depois para os stats de ImageNet
        x_unnorm = (x * 0.5) + 0.5 # De [-1,1] para [0,1]
        x_transformed = self.transform_input(x_unnorm) # Aplica resize e normalização ImageNet
        return self.features(x_transformed)

# Instância global do extrator de features
# Iniciaremos apenas UMA vez no run_evolution
inception_feature_extractor = None

def discriminator_score_from_image(discriminator, image_np, device='cpu'):
    arr = np.asarray(image_np).astype(np.float32) / 255.0
    if arr.ndim == 2:
        arr = arr[None, None, ...]
    else:
        arr = arr.transpose(2,0,1)[None, ...]

    tensor = torch.from_numpy(arr).to(device).float()
    tensor = (tensor - 0.5) / 0.5 # Normaliza de [0, 1] para [-1, 1], igual ao treinamento

    with torch.no_grad():
        score = discriminator(tensor).cpu().item()
    return score

# --- Nova métrica de diversidade baseada em features ---
def feature_diversity_metric(feature_extractor, image_np, device='cpu'):
    # arr: (H,W,C) uint8
    arr = np.asarray(image_np).astype(np.float32) / 255.0
    # Adiciona dimensão de batch e permuta para (1, C, H, W)
    if arr.ndim == 2:
        arr = arr[None, None, ...]
    else:
        arr = arr.transpose(2,0,1)[None, ...]

    tensor = torch.from_numpy(arr).to(device).float()

    with torch.no_grad():
        # Extrai features e calcula a variância entre as features
        # Uma imagem única, então a "diversidade" é a complexidade das suas próprias features
        features = feature_extractor(tensor)
        # Podemos usar a norma L2 ou a variância das features
        # A variância ainda é uma boa heurística para "complexidade interna"
        return float(features.std().cpu().item())
        # Ou um valor baseado em norma, que pode ser interpretado como "distintividade"
        # return float(torch.norm(features, p=2).cpu().item())

# -----------------------------
# 6) Definição do problema para pymoo
# -----------------------------
class CPPN_Evolution_Problem(Problem):
    def __init__(self, genome_size, discriminator, cppn_template, size=64, out_channels=3, device='cpu', feature_extractor=None):
        super().__init__(n_var=genome_size, n_obj=2, n_constr=0, xl=-5.0, xu=5.0)
        self.discriminator = discriminator
        self.cppn_template = cppn_template
        self.size = size
        self.out_channels = out_channels
        self.device = device
        self.feature_extractor = feature_extractor # Novo: para diversidade

    def _evaluate(self, X, out, *args, **kwargs):
        pop_size = X.shape[0]
        f1 = np.zeros((pop_size,))
        f2 = np.zeros((pop_size,))

        for i in range(pop_size):
            genome = X[i]
            cppn = copy.deepcopy(self.cppn_template)
            unpack_vector_to_model(cppn, genome)
            cppn.to(self.device)
            img = cppn_generate_image(cppn, size=self.size, out_channels=self.out_channels, device=self.device)

            try:
                score = discriminator_score_from_image(self.discriminator, img, device=self.device)
            except Exception:
                score = 0.0

            # --- Alteração: Usar a nova métrica de diversidade ---
            if self.feature_extractor:
                div = feature_diversity_metric(self.feature_extractor, img, device=self.device)
            else:
                div = diversity_metric(img) # fallback para a métrica simples
            # ----------------------------------------------------

            f1[i] = -score
            f2[i] = -div

        out['F'] = np.column_stack([f1, f2])

# -----------------------------
# 7) Executor/Runner - ALTERADO PARA INCLUIR NOVIDADES
# -----------------------------
def run_evolution(pop_size=20, generations=30, img_size=64, out_channels=3, device='cpu'):

    # Caminho para salvar o discriminador treinado (usando o Drive, conforme discutimos)
    DRIVE_BASE_PATH = "/content/drive/My Drive/CPPN_Project_Advanced/" # Nova pasta para evitar conflito
    os.makedirs(DRIVE_BASE_PATH, exist_ok=True)
    DISCRIMINATOR_PATH = os.path.join(DRIVE_BASE_PATH, "vgg_discriminator_wikiart.pth")

    # --- 0. Inicializar o extrator de features do Inception para diversidade ---
    global inception_feature_extractor
    if inception_feature_extractor is None:
        print("Carregando Feature Extractor (InceptionV3) para métrica de diversidade...")
        inception_feature_extractor = FeatureExtractor(model_name='inception_v3', device=device)
        print("Feature Extractor carregado.")

    # 1) criar template CPPN (agora mais complexa)
    cppn_template = CPPN(out_channels=out_channels, hidden_size=32).to(device) # Aumentado hidden_size
    genome_size = count_params(cppn_template)
    print(f"Genome size: {genome_size} (para {out_channels} canais, CPPN mais complexa)")

    # 2) criar discriminador (agora VGGDiscriminator pré-treinado)
    discriminator = VGGDiscriminator(in_channels=out_channels).to(device)

    # 3) TREINAR ou CARREGAR o discriminador
    if os.path.exists(DISCRIMINATOR_PATH):
        print(f"Carregando discriminador pré-treinado de {DISCRIMINATOR_PATH}...")
        discriminator.load_state_dict(torch.load(DISCRIMINATOR_PATH))
    else:
        print("Nenhum discriminador pré-treinado encontrado. Iniciando treinamento do VGGDiscriminator...")
        discriminator = train_discriminator(discriminator,
                                            cppn_template,
                                            device,
                                            img_size=img_size,
                                            out_channels=out_channels,
                                            batch_size=32, # Batch size pode precisar de ajuste para VGG
                                            epochs=10, # Mais épocas para o discriminador mais complexo
                                            drive_base_path=DRIVE_BASE_PATH)

        torch.save(discriminator.state_dict(), DISCRIMINATOR_PATH)
        print(f"Discriminador treinado e salvo em {DISCRIMINATOR_PATH}")

    discriminator.eval()

    # 4) definir problema (passando o feature_extractor)
    problem = CPPN_Evolution_Problem(genome_size=genome_size,
                                     discriminator=discriminator,
                                     cppn_template=cppn_template,
                                     size=img_size,
                                     out_channels=out_channels,
                                     device=device,
                                     feature_extractor=inception_feature_extractor) # Passa o extrator aqui

    # 5) algoritmo NSGA-II
    algorithm = NSGA2(pop_size=pop_size,
                      # Adicionado: Operadores genéticos mais flexíveis para maior exploração
                      sampling=pymoo.operators.sampling.LHS(),
                      crossover=pymoo.operators.crossover.SPX(), # Single-Point Crossover
                      mutation=pymoo.operators.mutation.PM(eta=20.0, prob_var=0.5)) # Polynomial Mutation

    termination = get_termination("n_gen", generations)

    print("\nIniciando evolução (NSGA-II) com configurações avançadas...")
    res = minimize(problem,
                   algorithm,
                   termination,
                   seed=42, # Usar um seed para reprodutibilidade
                   save_history=False,
                   verbose=True)

    return res

# -----------------------------
# 8) Exemplo de uso (main)
# -----------------------------
if __name__ == '__main__':
    # Montar Google Drive (se estiver no Colab)
    # from google.colab import drive
    # drive.mount('/content/drive')

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print('Device:', device)

    # Configurações para potencialmente mais complexidade
    POP = 40 # Aumentar população
    GENS = 50 # Aumentar gerações
    IMG_SIZE = 128 # Aumentar o tamanho da imagem para mais detalhes
    OUT_CH = 3

    res = run_evolution(pop_size=POP, generations=GENS, img_size=IMG_SIZE, out_channels=OUT_CH, device=device)

    # salvar as primeiras imagens da população final
    os.makedirs('out_evolved_advanced', exist_ok=True) # Nova pasta de saída
    for i, genome in enumerate(res.X[:20]): # Salvar mais imagens
        cppn_template = CPPN(out_channels=OUT_CH, hidden_size=32)
        unpack_vector_to_model(cppn_template, genome)
        img = cppn_generate_image(cppn_template, size=IMG_SIZE, out_channels=OUT_CH, device=device)
        im = Image.fromarray(img)
        im.save(f'out_evolved_advanced/ind_{i}.png')

    print(f'Evolução finalizada. Imagens salvas em ./out_evolved_advanced/ (tamanho {IMG_SIZE}x{IMG_SIZE})')

# -----------------------------
# Observações finais (atualizadas)
# -----------------------------
# - As imagens ainda serão "abstratas" na natureza, mas as melhorias visam
#   padrões mais complexos e que se alinhem melhor com features de imagens reais.
# - O treino do discriminador VGG agora é mais exigente.
# - AUMENTAR IMG_SIZE AUMENTA MUITO O TEMPO DE GERAÇÃO E AVALIAÇÃO!
#   Considere rodar com IMG_SIZE=64 inicialmente para testes.
# - A métrica de diversidade baseada em InceptionV3 é mais robusta.
# - Novas pastas para arquivos salvos: CPPN_Project_Advanced e out_evolved_advanced.

Device: cpu
Carregando Feature Extractor (InceptionV3) para métrica de diversidade...
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


100%|██████████| 104M/104M [00:07<00:00, 13.7MB/s]


Feature Extractor carregado.
Genome size: 2307 (para 3 canais, CPPN mais complexa)
Downloading: "https://download.pytorch.org/models/vgg11-8a719046.pth" to /root/.cache/torch/hub/checkpoints/vgg11-8a719046.pth


100%|██████████| 507M/507M [00:05<00:00, 102MB/s]


Nenhum discriminador pré-treinado encontrado. Iniciando treinamento do VGGDiscriminator...
Iniciando pré-treinamento do Discriminador...
Carregando WikiArt (pode demorar na primeira vez)...


...

data/train-00071-of-00072.parquet:   0%|          | 0.00/367M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/81444 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/45 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch [1/10], Batch [1/2546], Loss D: 1.2618 (Real: 0.7468, Fake: 0.5150)
Epoch [1/10], Batch [101/2546], Loss D: 0.0000 (Real: 0.0000, Fake: 0.0000)
Epoch [1/10], Batch [201/2546], Loss D: 0.0000 (Real: 0.0000, Fake: 0.0000)
Epoch [1/10], Batch [301/2546], Loss D: 0.0000 (Real: 0.0000, Fake: 0.0000)
Epoch [1/10], Batch [401/2546], Loss D: 0.0000 (Real: 0.0000, Fake: 0.0000)
Epoch [1/10], Batch [501/2546], Loss D: 0.0000 (Real: 0.0000, Fake: 0.0000)
Epoch [1/10], Batch [601/2546], Loss D: 0.0000 (Real: 0.0000, Fake: 0.0000)
Epoch [1/10], Batch [701/2546], Loss D: 0.0000 (Real: 0.0000, Fake: 0.0000)
Epoch [1/10], Batch [801/2546], Loss D: 0.0000 (Real: 0.0000, Fake: 0.0000)
Epoch [1/10], Batch [901/2546], Loss D: 0.0000 (Real: 0.0000, Fake: 0.0000)
Epoch [1/10], Batch [1001/2546], Loss D: 0.0000 (Real: 0.0000, Fake: 0.0000)
Epoch [1/10], Batch [1101/2546], Loss D: 0.0000 (Real: 0.0000, Fake: 0.0000)
Epoch [1/10], Batch [1201/2546], Loss D: 0.0000 (Real: 0.0000, Fake: 0.0000)
Epoch [1/10